<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
#!pip install polars

In [1]:
import polars as pl
import random
import string


# Generacion de Datos

In [5]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""fzbqk""",17,79.515669,"2020-08-03 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2897
2,"""ckmfu""",5,33.515012,"2016-12-08 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4173
3,"""mzbhz""",2,54.801953,"2022-10-15 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4399
4,"""tsnba""",10,42.424052,"2022-07-26 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",4028
5,"""qlehu""",2,69.904942,"2016-05-19 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",2440


In [6]:
df = df.lazy()
df

# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

In [7]:
lazy_df = df.lazy()

# Realizar las operaciones de limpieza y manipulación
resultado = (
    lazy_df
    .filter(pl.col("cantidad") >= 5)
    .with_columns((pl.col("cantidad") * pl.col("precio_unitario")).alias("total_venta"))
   
    .drop("precio_unitario")
    .group_by("producto")
    .agg([
        pl.sum("total_venta").alias("total_vendido"),
        pl.sum("cantidad").alias("cantidad_total_vendida"),
        
    ])
    .sort("total_vendido", descending = True)
)

  

# Ejecutar y mostrar los resultados
resultado_df = resultado.collect()
print(resultado_df)

shape: (793, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ zqnig    ┆ 1997.506632   ┆ 20                     │
│ qrmkn    ┆ 1945.266604   ┆ 20                     │
│ wyzlx    ┆ 1910.527558   ┆ 20                     │
│ fbiqe    ┆ 1884.67713    ┆ 19                     │
│ lywox    ┆ 1883.934981   ┆ 19                     │
│ …        ┆ …             ┆ …                      │
│ wlvng    ┆ 84.407189     ┆ 6                      │
│ fkztc    ┆ 79.945161     ┆ 7                      │
│ amcgd    ┆ 78.072966     ┆ 6                      │
│ cjtmj    ┆ 76.536581     ┆ 6                      │
│ araqu    ┆ 51.605197     ┆ 5                      │
└──────────┴───────────────┴────────────────────────┘


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [8]:
# Tu codigo aqui
resultado_df = resultado.collect()
resultado_df.write_parquet("resultado.parquet")

# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [9]:
# Tu codigo aqui
df = pl.read_parquet("resultado.parquet")
print(df.head())

shape: (5, 3)
┌──────────┬───────────────┬────────────────────────┐
│ producto ┆ total_vendido ┆ cantidad_total_vendida │
│ ---      ┆ ---           ┆ ---                    │
│ str      ┆ f64           ┆ i64                    │
╞══════════╪═══════════════╪════════════════════════╡
│ zqnig    ┆ 1997.506632   ┆ 20                     │
│ qrmkn    ┆ 1945.266604   ┆ 20                     │
│ wyzlx    ┆ 1910.527558   ┆ 20                     │
│ fbiqe    ┆ 1884.67713    ┆ 19                     │
│ lywox    ┆ 1883.934981   ┆ 19                     │
└──────────┴───────────────┴────────────────────────┘
